## Søke etter en referanse på en gitt dato og liste opp innholdet i xml-fila

In [ ]:
from google.cloud import storage
from dapla import AuthClient
import xml.etree.ElementTree as ET
import xmltodict

def parse_node(node, ancestor_string=""):
    #print(type(node), dir(node))

    if ancestor_string:
        node_string = ".".join([ancestor_string, node.tag])
    else:
        node_string = node.tag
    tag_list = [node_string]
    text = node.text
    if text:
        text_list = [text.strip()]
    else:
        text_list = [""]
    attr_list = [node.attrib]
    for child_node in list(node):
        child_tag_list, child_text_list, child_attr_list = parse_node(child_node, ancestor_string=node_string)
        tag_list.extend(child_tag_list)
        text_list.extend(child_text_list)
        attr_list.extend(child_attr_list)
    return tag_list, text_list, attr_list


def parse_xml(project, bucket_name, filename):
    storage_client = storage.client.Client(credentials=AuthClient.fetch_google_credentials(), project=project,)
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.get_blob(filename)
    #print ("Filnavn: ", blob.name)

    read_output = blob.download_as_string()
    tree  = ET.ElementTree(ET.fromstring(read_output))

    root_node = tree.getroot()
    tags, texts, attrs = parse_node(root_node)

    y = 0
    for x in tags:
        print(tags[y], ': ', texts[y])
        y = y + 1

        
def find_reference_date(project, bucket_name, folder, reference):

    try:
        storage_client = storage.client.Client(credentials=AuthClient.fetch_google_credentials(), project=project,)
        bucket = storage_client.get_bucket(bucket_name)
        blobs = bucket.list_blobs(prefix=folder)

        # Loops thru the files in the folder
        for blob in blobs:

            # Jumps over the folder itself
            if blob.name == folder:
                continue

            if reference in blob.name and '.xml' in blob.name:
                print ("Filnavn:", blob.name)
                FILENAME = blob.name
                parse_xml(PROJECT, BUCKET, FILENAME)
                print ('')

    except Exception as e:
        print(f'Unexpected exception: {e}')
        raise e

In [ ]:
PROJECT               = "altinn-data-staging-c629"
BUCKET                = "ra0678-01-altinn-data-staging-c629-ssb-altinn"
DATE                  = "2023/3/2"
REFERENCE             = "5fb36dae75f9"

find_reference_date(PROJECT, BUCKET, DATE, REFERENCE)